READ CSV, AND EXTRACT ALL STREAM INFORMATION FOR EACH WALLET ADDRESS
receives wallets_with_allocations table and streams table. 
outputs json file with all infor for each stream.

In [5]:
import csv
import json
import os

# File paths
input_csv_path = 'wallet_amount.csv'  # Path to input CSV file with wallet amounts
lookup_csv_path = 'token_dist.csv'    # Path to lookup CSV file with token distribution data
output_json_path = 'all_stream_info.json'  # Path to output JSON file

# Read lookup CSV into a dictionary
lookup_dict = {}  # Initialize an empty dictionary for lookup data
with open(lookup_csv_path, mode='r', encoding='utf-8-sig') as lookup_csv:  # Open lookup CSV file
    reader = csv.DictReader(lookup_csv)  # Create a CSV dictionary reader
    for row in reader:  # Iterate over each row in the CSV
        lookup_dict[row["Allocation (Vesting) Category in Our Database"]] = row  # Map category to row data

# Fields to remove from extracted info
fields_to_remove = [  # List of fields to be excluded from extracted data
    "#",
    "Allocation (Vesting) Category in Our Database",
    "Allocation Main Category",
    "Allocation Sub Category",
    "Source of the Distribution"
]

# Read input CSV and process
output_data = []  # Initialize an empty list for the output data
with open(input_csv_path, mode='r', encoding='utf-8-sig') as input_csv:  # Open input CSV file
    reader = csv.DictReader(input_csv)  # Create a CSV dictionary reader
    for row in reader:  # Iterate over each row in the CSV
        category = row["Allocation (Vesting) Category in Our Database"]  # Get the category from the current row
        if category in lookup_dict:  # Check if the category exists in the lookup dictionary
            # Extract relevant information excluding specified fields
            extracted_info = {k: v for k, v in lookup_dict[category].items() if k not in fields_to_remove}
            # Append the combined data to output_data list
            output_data.append({
                "Wallet Address": row["Wallet Address"],  # Add wallet address from input CSV
                "Allocated Amount": row["Allocated Amount"],  # Add allocated amount from input CSV
                **extracted_info  # Add extracted information from lookup CSV
            })

# Write output to JSON
with open(output_json_path, mode='w', encoding='utf-8') as output_json:  # Open output JSON file
    json.dump(output_data, output_json, indent=4)  # Write the output data to JSON file with indentation

print(f"Data has been written to {output_json_path}")  # Print confirmation message


Data has been written to all_stream_info.json


GIVEN THE STREAM TYPE, CREATE THE APPROPRIATE SMART CONTRACT. DIVIDE THEM BY FOLDERS

In [4]:
import json
import time
from datetime import datetime
import os

# Constants for token and lockup addresses
TOKEN = '0x6B175474E89094C44Da98b954EedeAC495271d0F'
LOCKUP_LINEAR = '0xFCF737582d167c7D20A336532eb8BCcA8CF8e350'
LOCKUP_DYNAMIC = '0x461E13056a3a3265CEF4c593F01b2e960755dE91'

def convert_to_unix(date_str):
    """
    Convert a date string to a Unix timestamp.
    
    Args:
    date_str (str): The date string in the format "dd Month yyyy".
    
    Returns:
    int: The Unix timestamp.
    """
    return int(time.mktime(datetime.strptime(date_str.strip(), "%d %B %Y").timetuple()))

def generate_contracts_from_file(input_file):
    """
    Generate contract code from a JSON file.
    
    Args:
    input_file (str): Path to the JSON input file.
    
    Returns:
    list: A list of tuples containing contract type and contract code.
    """
    with open(input_file, 'r') as file:  # Open the input JSON file
        json_data = json.load(file)  # Load the JSON data

    contracts = []  # Initialize an empty list to store contracts

    for i, entry in enumerate(json_data):  # Iterate over each entry in the JSON data
        # Extract and calculate various parameters
        cliff_months = int(entry["Cliff (Months)"])
        cliff_duration = int((cliff_months * 30.44 * 24 * 60 * 60))  # Approximate month duration in seconds
        start_unix = convert_to_unix(entry["Start Date"])
        cliff_unix = start_unix + cliff_duration
        end_unix = convert_to_unix(entry["End Date"])
        total_amount = int(entry["Allocated Amount"].replace(",", ""))
        tge_perc = int(entry['Allocation on TGE'].replace("%", ""))/100

        if entry["Name of the Stream on Sablier"].lower() == "cliff stream":
            # Create contract code for linear lockup
            contract_linear_lockup = f"""\
            // SPDX-License-Identifier: GPL-3.0-or-later
            pragma solidity >=0.8.19;

            import {{ IERC20 }} from "@openzeppelin/contracts/token/ERC20/IERC20.sol";
            import {{ ud60x18 }} from "@prb/math/src/UD60x18.sol";
            import {{ ISablierV2LockupLinear }} from "@sablier/v2-core/src/interfaces/ISablierV2LockupLinear.sol";
            import {{ Broker, LockupLinear }} from "@sablier/v2-core/src/types/DataTypes.sol";

            contract LockupLinearStreamCreator {{
                // Mainnet addresses
                IERC20 public constant TOKEN = IERC20({TOKEN});
                ISablierV2LockupLinear public constant LOCKUP_LINEAR =
                    ISablierV2LockupLinear({LOCKUP_LINEAR});

                function createStream(uint128 totalAmount) public returns (uint256 streamId) {{
                    // Transfer the provided amount of TOKEN tokens to this contract
                    TOKEN.transferFrom(msg.sender, address(this), totalAmount);

                    // Approve the Sablier contract to spend TOKEN
                    TOKEN.approve(address(LOCKUP_LINEAR), totalAmount);

                    // Declare the params struct
                    LockupLinear.CreateWithRange memory params;

                    // Declare the function parameters
                    params.sender = msg.sender; // The sender will be able to cancel the stream
                    params.recipient = address({entry["Wallet Address"]}); // The recipient of the streamed assets
                    params.totalAmount = {total_amount}; // Total amount is the amount inclusive of all fees
                    params.asset = TOKEN; // The streaming asset
                    params.cancelable = {"true" if entry["Cancelability"].lower() == "on" else "false"}; // Whether the stream will be cancelable or not
                    params.transferable = {"true" if entry["Transferability"].lower() == "on" else "false"}; // Whether the stream will be transferable or not
                    params.range = LockupLinear.Range({{
                        start: {start_unix}, // Assets will be unlocked from this time
                        cliff: {cliff_unix}, // End of cliffing period
                        end: {end_unix} // End time in Unix timestamp
                    }});

                    streamId = LOCKUP_LINEAR.createWithRange(params);
                }}
            }}
            """
            contracts.append(('linear_lockup', contract_linear_lockup))

        elif entry["Name of the Stream on Sablier"].lower() == "linear stream":
            # Create contract code for time lock
            contract_time_lock = f"""\
            // SPDX-License-Identifier: GPL-3.0-or-later
            pragma solidity >=0.8.19;

            import {{ IERC20 }} from "@openzeppelin/contracts/token/ERC20/IERC20.sol";
            import {{ ud2x18 }} from "@prb/math/src/UD2x18.sol";
            import {{ ud60x18 }} from "@prb/math/src/UD60x18.sol";
            import {{ ISablierV2LockupDynamic }} from "@sablier/v2-core/src/interfaces/ISablierV2LockupDynamic.sol";
            import {{ Broker, LockupDynamic }} from "@sablier/v2-core/src/types/DataTypes.sol";

            contract LockupDynamicCurvesCreator {{
                // Mainnet addresses
                IERC20 public constant TOKEN = IERC20({TOKEN});
                ISablierV2LockupDynamic public constant LOCKUP_DYNAMIC =
                    ISablierV2LockupDynamic({LOCKUP_DYNAMIC});

                function createStream_Timelock() external returns (uint256 streamId) {{
                    uint128 totalAmount = {total_amount};

                    // Transfer the provided amount of TOKEN tokens to this contract
                    TOKEN.transferFrom(msg.sender, address(this), totalAmount);

                    // Approve the Sablier contract to spend TOKEN
                    TOKEN.approve(address(LOCKUP_DYNAMIC), totalAmount);

                    // Declare the params struct
                    LockupDynamic.CreateWithMilestones memory params;

                    // Declare the function parameters
                    params.sender = msg.sender; // The sender will be able to cancel the stream
                    params.recipient = {entry["Wallet Address"]}; // The recipient of the streamed assets
                    params.startTime = {start_unix}; // Start time in Unix timestamp
                    params.totalAmount = totalAmount; // Total amount is the amount inclusive of all fees
                    params.asset = TOKEN; // The streaming asset
                    params.cancelable = {"true" if entry["Cancelability"].lower() == "on" else "false"}; // Whether the stream will be cancelable or not
                    params.transferable = {"true" if entry["Transferability"].lower() == "on" else "false"}; // Whether the stream will be transferable or not

                    // Declare a two-size segment to match the curve shape
                    params.segments = new LockupDynamic.Segment ;
                    params.segments[0] = LockupDynamic.Segment({{
                        amount: 0, 
                        exponent: ud2x18(1e18),
                        milestone: {start_unix + 1} 
                    }});
                    params.segments[1] = LockupDynamic.Segment({{
                        amount: {total_amount}, 
                        exponent: ud2x18(1e18),
                        milestone: {start_unix + 2} 
                        
                    }});

                    // Create the LockupDynamic stream
                    streamId = LOCKUP_DYNAMIC.createWithMilestones(params);
                }}
            }}
            """
            contracts.append(('time_lock', contract_time_lock))

        elif entry["Name of the Stream on Sablier"].lower() == "unlock-linear":
            # Create contract code for unlock linear
            contract_unlock_linear = f"""\
            // SPDX-License-Identifier: GPL-3.0-or-later
            pragma solidity >=0.8.19;

            import {{ IERC20 }} from "@openzeppelin/contracts/token/ERC20/IERC20.sol";
            import {{ ud2x18 }} from "@prb/math/src/UD2x18.sol";
            import {{ ud60x18 }} from "@prb/math/src/UD60x18.sol";
            import {{ ISablierV2LockupDynamic }} from "@sablier/v2-core/src/interfaces/ISablierV2LockupDynamic.sol";
            import {{ Broker, LockupDynamic }} from "@sablier/v2-core/src/types/DataTypes.sol";

            contract LockupDynamicCurvesCreator {{
                // Mainnet addresses
                IERC20 public constant TOKEN = IERC20({TOKEN});
                ISablierV2LockupDynamic public constant LOCKUP_DYNAMIC =
                    ISablierV2LockupDynamic({LOCKUP_DYNAMIC});

                function createStream_UnlockLinear() external returns (uint256 streamId) {{
                    uint128 totalAmount = {total_amount};

                    // Transfer the provided amount of TOKEN tokens to this contract
                    TOKEN.transferFrom(msg.sender, address(this), totalAmount);

                    // Approve the Sablier contract to spend TOKEN
                    TOKEN.approve(address(LOCKUP_DYNAMIC), totalAmount);

                    // Declare the params struct
                    LockupDynamic.CreateWithMilestones memory params;

                    // Declare the function parameters
                    params.sender = msg.sender; // The sender will be able to cancel the stream
                    params.recipient = {entry["Wallet Address"]}; // The recipient of the streamed assets
                    params.startTime = {start_unix}; // Start time in Unix timestamp
                    params.totalAmount = totalAmount; // Total amount is the amount inclusive of all fees
                    params.asset = TOKEN; // The streaming asset
                    params.cancelable = {"true" if entry["Cancelability"].lower() == "on" else "false"}; // Whether the stream will be cancelable or not
                    params.transferable = {"true" if entry["Transferability"].lower() == "on" else "false"}; // Whether the stream will be transferable or not

                    // Declare a two-size segment to match the curve shape
                    params.segments = new LockupDynamic.Segment ;
                    params.segments[0] = LockupDynamic.Segment({{ 
                        amount: {int(total_amount*tge_perc)}, 
                        exponent: ud2x18(1e18), 
                        milestone: {start_unix + 1}
                    }});
                    params.segments[1] = LockupDynamic.Segment({{ 
                        amount: {int(total_amount-(total_amount*tge_perc))}, 
                        exponent: ud2x18(1e18),
                        milestone: {end_unix} }});

                    // Create the LockupDynamic stream
                    streamId = LOCKUP_DYNAMIC.createWithMilestones(params);
                }}
            }}
            """
            contracts.append(('unlock_linear', contract_unlock_linear))

    return contracts

def save_contracts_to_files(contracts):
    """
    Save generated contracts to files.
    
    Args:
    contracts (list): List of tuples containing contract type and contract code.
    """
    base_dir = "contracts"  # Directory to save the contracts
    if not os.path.exists(base_dir):  # Check if the base directory exists
        os.makedirs(base_dir)  # Create the base directory if it doesn't exist

    for contract_type, contract in contracts:  # Iterate over each contract
        contract_dir = os.path.join(base_dir, contract_type)  # Directory for the specific contract type
        if not os.path.exists(contract_dir):  # Check if the contract directory exists
            os.makedirs(contract_dir)  # Create the contract directory if it doesn't exist

        # Get the number of existing contracts in the folder to avoid overwriting
        contract_files = os.listdir(contract_dir)
        contract_count = len(contract_files)  # Count the number of existing contracts

        filename = f"{contract_type}_contract_{contract_count + 1}.sol"  # Create a unique filename
        with open(os.path.join(contract_dir, filename), 'w') as file:  # Open the file for writing
            file.write(contract)  # Write the contract code to the file

# Example usage
input_file = 'all_stream_info.json'  # Replace with your JSON file path
contracts = generate_contracts_from_file(input_file)  # Generate contracts from the input file
save_contracts_to_files(contracts)  # Save the generated contracts to files


CREATE BATCH LINEAR LOCK-UP

In [7]:
import os
import re
from web3 import Web3
import json

# Define the structure for Batch.CreateWithRange
class LockupLinearRange:
    """
    A class to represent the range for Lockup Linear contracts.
    
    Attributes:
    start (int): Start time in Unix timestamp.
    cliff (int): Cliff time in Unix timestamp.
    end (int): End time in Unix timestamp.
    """
    def __init__(self, start, cliff, end):
        self.start = start
        self.cliff = cliff
        self.end = end

class BatchCreateWithRange:
    """
    A class to represent the batch parameters for creating Lockup Linear contracts.
    
    Attributes:
    sender (str): Address of the sender.
    recipient (str): Address of the recipient.
    totalAmount (int): Total amount to be locked up.
    asset (str): Address of the asset.
    cancelable (bool): Whether the contract is cancelable.
    transferable (bool): Whether the contract is transferable.
    range (LockupLinearRange): Range object containing start, cliff, and end times.
    """
    def __init__(self, sender, recipient, totalAmount, asset, cancelable, transferable, range):
        self.sender = sender
        self.recipient = recipient
        self.totalAmount = totalAmount
        self.asset = asset
        self.cancelable = cancelable
        self.transferable = transferable
        self.range = range

# Function to parse Solidity file and extract parameters
def parse_solidity_file(file_path):
    """
    Parse a Solidity file to extract contract parameters.
    
    Args:
    file_path (str): Path to the Solidity file.
    
    Returns:
    dict: A dictionary containing extracted parameters.
    """
    with open(file_path, 'r') as file:  # Open the Solidity file
        content = file.read()  # Read the file content
    
    print(f"Parsing file: {file_path}")
    params = {}

    # Extract the parameters using regex
    try:
        params['sender'] = re.search(r'params\.sender\s*=\s*(.*);', content).group(1).strip()  # Extract sender
        print(f"Found sender: {params['sender']}")
        
        recipient_match = re.search(r'params\.recipient\s*=\s*address\((0x[0-9a-fA-F]+)\);', content)  # Extract recipient
        if recipient_match:
            params['recipient'] = recipient_match.group(1)
            print(f"Found recipient: {params['recipient']}")
        else:
            raise ValueError("Could not match the recipient in the Solidity file.")
        
        params['totalAmount'] = int(re.search(r'params\.totalAmount\s*=\s*(.*);', content).group(1).strip())  # Extract total amount
        print(f"Found totalAmount: {params['totalAmount']}")
        
        asset_match = re.search(r'IERC20\((0x[0-9a-fA-F]+)\)', content)  # Extract asset
        if asset_match:
            params['asset'] = asset_match.group(1)
            print(f"Found asset: {params['asset']}")
        else:
            raise ValueError("Could not match the asset in the Solidity file.")
        
        params['cancelable'] = re.search(r'params\.cancelable\s*=\s*(.*);', content).group(1).strip().lower() == 'true'  # Extract cancelable
        print(f"Found cancelable: {params['cancelable']}")
        
        params['transferable'] = re.search(r'params\.transferable\s*=\s*(.*);', content).group(1).strip().lower() == 'true'  # Extract transferable
        print(f"Found transferable: {params['transferable']}")
        
        # Extract range parameters directly
        start_match = re.search(r'start:\s*(\d+)', content)  # Extract start time
        cliff_match = re.search(r'cliff:\s*(\d+)', content)  # Extract cliff time
        end_match = re.search(r'end:\s*(\d+)', content)  # Extract end time
        
        if start_match and cliff_match and end_match:
            params['range'] = LockupLinearRange(
                int(start_match.group(1)),
                int(cliff_match.group(1)),
                int(end_match.group(1))
            )
            print(f"Found range: start={params['range'].start}, cliff={params['range'].cliff}, end={params['range'].end}")
        else:
            raise ValueError("Could not match the range parameters in the Solidity file.")
    except AttributeError as e:
        print(f"Error parsing file {file_path}: {e}")
        raise

    return params

# Function to create batch of CreateWithRange objects
def create_batch_from_files(files):
    """
    Create a batch of BatchCreateWithRange objects from a list of Solidity files.
    
    Args:
    files (list): List of paths to Solidity files.
    
    Returns:
    list: A list of BatchCreateWithRange objects.
    """
    batch = []
    for file_path in files:  # Iterate over each file path
        try:
            params = parse_solidity_file(file_path)  # Parse the Solidity file
            batch.append(BatchCreateWithRange(
                sender=params['sender'],
                recipient=params['recipient'],
                totalAmount=params['totalAmount'],
                asset=params['asset'],
                cancelable=params['cancelable'],
                transferable=params['transferable'],
                range=params['range']
            ))  # Append the BatchCreateWithRange object to the batch
        except ValueError as e:
            print(f"Skipping file {file_path} due to error: {e}")
    return batch

# Directory containing Solidity files
solidity_files_directory = 'contracts/linear_lockup/'

# Get list of all Solidity files in the directory
solidity_files = [os.path.join(solidity_files_directory, f) for f in os.listdir(solidity_files_directory) if f.endswith('.sol')]

# Create batch from Solidity files
batch = create_batch_from_files(solidity_files)

# Example of how to pass the batch to the createWithRange function
for b in batch:  # Print each batch item
    print(f"Sender: {b.sender}, Recipient: {b.recipient}, TotalAmount: {b.totalAmount}, Asset: {b.asset}, Cancelable: {b.cancelable}, Transferable: {b.transferable}, Range: ({b.range.start}, {b.range.cliff}, {b.range.end})")

# Connect to the Base blockchain using QuickNode
web3 = Web3(Web3.HTTPProvider('https://wild-nameless-sheet.base-sepolia.quiknode.pro/6fbb48054c8a584d38a44ffbcef70f6e78ccd53d/'))

# Load contract ABI from a file
with open('abi.json', 'r') as abi_file:  # Open the ABI file
    contract_abi = json.load(abi_file)  # Load the contract ABI

# Contract address
contract_address = '0x94E596EEd73b4e3171c067f05A87AB0268cA993c'

# Instantiate the contract
contract = web3.eth.contract(address=contract_address, abi=contract_abi)

# Account details
account = web3.toChecksumAddress('0x389431E8Bc3a5159895dc95D91C34A3457089591')
private_key = '12f23a131783385a50219e2e473218362acda165ac5f6d96ad1442722c066a71'

# Verify if the function exists
if hasattr(contract.functions, 'createWithRange'):
    print("The function createWithRange exists in the contract.")
else:
    print("The function createWithRange does not exist in the contract.")
    exit(1)

# Function to call createWithRange
def call_create_with_range(batch, contract):
    """
    Call the createWithRange function on the contract with the given batch of parameters.
    
    Args:
    batch (list): List of BatchCreateWithRange objects.
    contract (Contract): The instantiated contract object.
    
    Returns:
    dict: Transaction receipt.
    """
    # Prepare batch data
    batch_data = [{
        'sender': item.sender,
        'recipient': item.recipient,
        'totalAmount': item.totalAmount,
        'asset': item.asset,
        'cancelable': item.cancelable,
        'transferable': item.transferable,
        'range': {
            'start': item.range.start,
            'cliff': item.range.cliff,
            'end': item.range.end
        }
    } for item in batch]

    # Build the transaction
    nonce = web3.eth.getTransactionCount(account)  # Get the transaction count for the account
    txn = contract.functions.createWithRange(
        web3.toChecksumAddress('0x94E596EEd73b4e3171c067f05A87AB0268cA993c'),  # lockupLinear address
        web3.toChecksumAddress(batch[0].asset),  # Assuming all assets are the same
        batch_data
    ).buildTransaction({
        'chainId': 84532,
        'gas': 2000000,
        'gasPrice': web3.toWei('50', 'gwei'),
        'nonce': nonce,
    })

    # Sign the transaction
    signed_txn = web3.eth.account.signTransaction(txn, private_key=private_key)  # Sign the transaction

    # Send the transaction
    tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)  # Send the raw transaction

    # Wait for the transaction receipt
    receipt = web3.eth.waitForTransactionReceipt(tx_hash)  # Wait for the transaction receipt
    return receipt

print(dir(contract))

# Call the function
receipt = call_create_with_range(batch, contract)
print(f'Transaction receipt: {receipt}')


Parsing file: contracts/linear_lockup/linear_lockup_contract_1.sol
Found sender: msg.sender
Found recipient: 0x03ae395d04D1dE1A34F1bF6Ef86Ac53D9b35D41a
Found totalAmount: 2000000
Found asset: 0x6B175474E89094C44Da98b954EedeAC495271d0F
Found cancelable: True
Found transferable: False
Found range: start=1722466800, cliff=1754026992, end=1818370800
Sender: msg.sender, Recipient: 0x03ae395d04D1dE1A34F1bF6Ef86Ac53D9b35D41a, TotalAmount: 2000000, Asset: 0x6B175474E89094C44Da98b954EedeAC495271d0F, Cancelable: True, Transferable: False, Range: (1722466800, 1754026992, 1818370800)


CREATE BATCH TIME-LOCK/UNLOCK-LINEAR

In [ ]:
import os
import re

# Define the structure for Batch.CreateWithMilestones
class LockupDynamicSegment:
    """
    A class to represent a segment in Lockup Dynamic contracts.

    Attributes:
    amount (int): The amount for the segment.
    exponent (int): The exponent value.
    milestone (int): The milestone time in Unix timestamp.
    """
    def __init__(self, amount, exponent, milestone):
        self.amount = amount
        self.exponent = exponent
        self.milestone = milestone

class LockupDynamicCreateWithMilestones:
    """
    A class to represent the batch parameters for creating Lockup Dynamic contracts.

    Attributes:
    sender (str): Address of the sender.
    startTime (int): Start time in Unix timestamp.
    cancelable (bool): Whether the contract is cancelable.
    transferable (bool): Whether the contract is transferable.
    recipient (str): Address of the recipient.
    totalAmount (int): Total amount to be locked up.
    asset (str): Address of the asset.
    broker (str): Address of the broker (optional).
    segments (list): List of LockupDynamicSegment objects.
    """
    def __init__(self, sender, startTime, cancelable, transferable, recipient, totalAmount, asset, broker, segments):
        self.sender = sender
        self.startTime = startTime
        self.cancelable = cancelable
        self.transferable = transferable
        self.recipient = recipient
        self.totalAmount = totalAmount
        self.asset = asset
        self.broker = broker
        self.segments = segments

# Function to parse Solidity file and extract parameters
def parse_solidity_file(file_path):
    """
    Parse a Solidity file to extract contract parameters.

    Args:
    file_path (str): Path to the Solidity file.

    Returns:
    dict: A dictionary containing extracted parameters.
    """
    with open(file_path, 'r') as file:  # Open the Solidity file
        content = file.read()  # Read the file content
    
    print(f"Parsing file: {file_path}")
    params = {}

    try:
        # Extract the parameters using regex
        params['sender'] = re.search(r'params\.sender\s*=\s*(.*);', content).group(1).strip()  # Extract sender
        print(f"Found sender: {params['sender']}")
        
        recipient_match = re.search(r'params\.recipient\s*=\s*0x([0-9a-fA-F]+);', content)  # Extract recipient
        if recipient_match:
            params['recipient'] = '0x' + recipient_match.group(1)
            print(f"Found recipient: {params['recipient']}")
        else:
            raise ValueError("Could not match the recipient in the Solidity file.")
        
        startTime_match = re.search(r'params\.startTime\s*=\s*(\d+);', content)  # Extract start time
        if startTime_match:
            params['startTime'] = int(startTime_match.group(1).strip())
            print(f"Found startTime: {params['startTime']}")
        else:
            raise ValueError("Could not match the startTime in the Solidity file.")

        totalAmount_match = re.search(r'totalAmount\s*=\s*(\d+);', content)  # Extract total amount
        if totalAmount_match:
            params['totalAmount'] = int(totalAmount_match.group(1).strip())
            print(f"Found totalAmount: {params['totalAmount']}")
        else:
            raise ValueError("Could not match the totalAmount in the Solidity file.")
        
        asset_match = re.search(r'IERC20\((0x[0-9a-fA-F]+)\)', content)  # Extract asset
        if asset_match:
            params['asset'] = asset_match.group(1)
            print(f"Found asset: {params['asset']}")
        else:
            raise ValueError("Could not match the asset in the Solidity file.")
        
        params['cancelable'] = re.search(r'params\.cancelable\s*=\s*(.*);', content).group(1).strip().lower() == 'true'  # Extract cancelable
        print(f"Found cancelable: {params['cancelable']}")
        
        params['transferable'] = re.search(r'params\.transferable\s*=\s*(.*);', content).group(1).strip().lower() == 'true'  # Extract transferable
        print(f"Found transferable: {params['transferable']}")

        # Extract segments
        amount_matches = re.findall(r'amount:\s*([\d.]+(?:e\d+)?)', content)  # Extract amounts
        exponent_matches = re.findall(r'exponent:\s*ud2x18\(([\d.]+(?:e\d+)?)\)', content)  # Extract exponents
        milestone_matches = re.findall(r'milestone:\s*(\d+)', content)  # Extract milestones

        if len(amount_matches) >= 2 and len(exponent_matches) >= 2 and len(milestone_matches) >= 2:
            segments = []
            for i in range(2):
                amount = int(float(amount_matches[i]))  # Convert scientific notation to integer
                exponent = int(float(exponent_matches[i]))  # Convert scientific notation to integer
                milestone = int(milestone_matches[i])
                segments.append(LockupDynamicSegment(amount, exponent, milestone))
                print(f"Found segment {i}: amount={amount}, exponent={exponent}, milestone={milestone}")
            params['segments'] = segments
        else:
            raise ValueError("Could not match the segments in the Solidity file.")
    
    except AttributeError as e:
        print(f"Error parsing file {file_path}: {e}")
        raise

    return params

# Function to create batch of CreateWithMilestones objects
def create_batch_from_files(files):
    """
    Create a batch of LockupDynamicCreateWithMilestones objects from a list of Solidity files.

    Args:
    files (list): List of paths to Solidity files.

    Returns:
    list: A list of LockupDynamicCreateWithMilestones objects.
    """
    batch = []
    for file_path in files:  # Iterate over each file path
        try:
            params = parse_solidity_file(file_path)  # Parse the Solidity file
            batch.append(LockupDynamicCreateWithMilestones(
                sender=params['sender'],
                startTime=params['startTime'],
                cancelable=params['cancelable'],
                transferable=params['transferable'],
                recipient=params['recipient'],
                totalAmount=params['totalAmount'],
                asset=params['asset'],
                broker=None,  # Add broker extraction if needed
                segments=params['segments']
            ))  # Append the LockupDynamicCreateWithMilestones object to the batch
        except ValueError as e:
            print(f"Skipping file {file_path} due to error: {e}")
    return batch

# Directory containing Solidity files
solidity_files_directory = 'contracts/unlock_linear/'

# Get list of all Solidity files in the directory
solidity_files = [os.path.join(solidity_files_directory, f) for f in os.listdir(solidity_files_directory) if f.endswith('.sol')]

# Create batch from Solidity files
batch = create_batch_from_files(solidity_files)

# Example of how to pass the batch to the createWithMilestones function
for b in batch:  # Print each batch item
    print(f"Sender: {b.sender}, Recipient: {b.recipient}, StartTime: {b.startTime}, TotalAmount: {b.totalAmount}, Asset: {b.asset}, Cancelable: {b.cancelable}, Transferable: {b.transferable}")
    for i, segment in enumerate(b.segments):
        print(f"  Segment {i}: Amount: {segment.amount}, Exponent: {segment.exponent}, Milestone: {segment.milestone}")
